In [1]:
import json
import pandas as pd
search_engine = "brave"
model_name = 'llama3'

with open(f"/home/hanlv/workspace/data/machine_learning/dataset/research/misinformation_dataset/COVMIS-2024/train_{search_engine}_search.json", "r") as f:
    data_search = json.load(f)
with open("covmis_data/train.json", "r") as f:
    data_train = json.load(f)

def load_diff(model_name, diff_version):
    with open(f"output/diff_{model_name}_v{diff_version}.json", "r") as f:
        return json.load(f)

def load_train():
    with open("covmis_data/train.json", "r") as f:
        return json.load(f)
    
def item_exist(x, data):
    for item in data:
        if item["id"] == x["id"]:
            return True
    return False

def load_type1():
    with open("output/type_1.json", "r") as f:
        return json.load(f)
    
def load_type2():
    with open("output/type_2.json", "r") as f:
        return json.load(f)

def save_type1(x):
    with open("output/type_1.json", "w") as f:
        json.dump(x, f, indent=4)
        
def save_type2(x):
    with open("output/type_2.json", "w") as f:
        json.dump(x, f, indent=4)

def save_excel(file_name, ids, claims, labels, related_urls, preds=None, human_checks=None):

    if preds:
        df = pd.DataFrame(columns=["id", "claim", "label", "related_url", "prediction"])
        # data ={'claim': claims, 'label':labels, "related_url":related_urls, 'prediction':preds}
        for i, row in enumerate(claims):
            df.loc[i] = [ids[i], claims[i], labels[i], related_urls[i], preds[i]]
    elif human_checks:
        df = pd.DataFrame(columns=["id", "claim", "label", "related_url", "human_check"])
        # data ={'claim': claims, 'label':labels, "related_url":related_urls, 'human_check':human_checks}
        for i, row in enumerate(claims):
            df.loc[i] = [ids[i], claims[i], labels[i], related_urls[i], human_checks[i]]
    else:
        df = pd.DataFrame(columns=["id", "claim", "label", "related_url"])
        # data ={'claim': claims, 'label':labels, "related_url":related_urls}
        for i, row in enumerate(claims):
            df.loc[i] = [ids[i], claims[i], labels[i], related_urls[i]]
    df.to_excel(f'{file_name}.xlsx', index=False)


In [2]:
wrong_claims_dir = "../test_metric_single_llm/with_mixtral_info/brave/data1-split=0.5:9.5-ratio=1.0/lora-r=2/openchat-3.5-0106-lr=2e-4.txt"
with open(wrong_claims_dir) as f:
    wrong_claims = f.readlines()[2:]

wrong_labels = [s[s.find("Label: ") + len("Label: "): s.find(". Pred")].strip() for s in wrong_claims]
wrong_preds = [s[s.find("Pred: ") + len("Pred: "):s.find(". Claim")].strip() for s in wrong_claims]
wrong_claims = [s[s.find("Claim: ") + len("Claim: "):].strip() for s in wrong_claims]

len(wrong_claims)


FileNotFoundError: [Errno 2] No such file or directory: '../test_metric_single_llm/with_mixtral_info/brave/data1-split=0.5:9.5-ratio=1.0/lora-r=2/openchat-3.5-0106-lr=2e-4.txt'

模型分类错误的样例中不包含在先验知识与label不符的样例中的情况：如果这些样例均是label本身就正确，那么“先验知识与label不符”的策略可行

In [3]:
diff_version = 1
wrong_claims_diff = []
data_diff = load_diff("llama3", diff_version)

cnt = 0

check_claim_list = []
check_preds_list = []
check_labels_list = []

for item in data_diff:
    if item["match2"].lower() == "no":
        wrong_claims_diff.append(item["claim"])


for i, claim in enumerate(wrong_claims):
    if claim not in wrong_claims_diff: # 检查不在diff中的是不是都是对的
        check_claim_list.append(claim)
        check_preds_list.append(wrong_preds[i])
        check_labels_list.append(wrong_labels[i])

# save_excel("check_claim", check_claim_list, check_labels_list, check_preds_list)


先验知识与label不符的样例

In [2]:
diff_version = 1
wrong_claims_diff = []
labels_diff = []
related_urls_diff = []
ids_diff = []
# human_checks = []
data_diff = load_diff("llama3", diff_version)
data_type1 = load_type1()
data_type2 = load_type2()
false_claims_trump = pd.read_csv('wapo_trumpclaims_export-012021.csv')
claims = list(false_claims_trump.loc[:, "claim"])

for i, item in enumerate(data_diff):
    if item["match2"].lower() != "no":
        continue
    if item_exist(item, data_type1) or item_exist(item, data_type2):
        continue

    label = int(item["label"])
    need_pass = False

    for search_result_id in range(2):
        if len(data_search[i][f"{search_engine}_search_results"]["web"]["results"]) == search_result_id:
            break
        title_bak = data_search[i][f"{search_engine}_search_results"]["web"]["results"][search_result_id]["title"]
        title = title_bak
        url = data_search[i][f"{search_engine}_search_results"]["web"]["results"][search_result_id]["url"]
        check_list = ["poynter.org"] # 

        ok = False
        for site in check_list:
            if site in url:
                ok = True
                break
        if not ok:
            continue

        pos_st = title.find(item["claim"].split(' ')[0])
        if pos_st == -1:
            continue
        pos_en = len(title)
        if title.endswith(" ..."):
            pos_en -= 4
        elif title.endswith("..."):
            pos_en -= 3

        title = title[pos_st : pos_en]
        pos_en = len(title)

        #  - Poynter
        end_list = ["- Poynter", "-", ]
        for s_end in end_list:
            if title.endswith(s_end):
                pos_en -= len(s_end)
        
        if len(title.split()) >= 5 and title.strip() in item["claim"] and title_bak.lower().startswith("false:"):
            if label == 0:
                need_pass = True
                break
    if need_pass:
        # print(item["claim"])
        continue
    
    wrong_claims_diff.append(item["claim"])
    related_urls_diff.append(data_train[i]["related_url"])
    ids_diff.append(item["id"])

    if label == 0:
        label = "FALSE"
    elif label == 2:
        label = "TRUE"
    else:
        raise Exception()
    labels_diff.append(label)

    # human_check = ""
    # for x in claims:
    #     if ("Donald Trump" in data_search[i]["claimant"] or "Donald J. Trump" in data_search[i]["claimant"]) and item["claim"] in x: # 还是要检查下是不是川普说的，不然一些不完整的都会算进去
    #         human_check = "FALSE"
    #         break

    # human_checks.append(human_check)
    
# 属于false claims 的标注
# save_excel("diff_llama3_v99", ids_diff, wrong_claims_diff, labels_diff, related_urls_diff, human_checks=human_checks)


In [4]:
len(labels_diff)


1517

In [ ]:
websites = [
    'politifact.com/factchecks', 'snopes.com/fact-check', 'factcheck.afp', 'africacheck', 
    'polygraph.info', 
    # 'reuters',  # 
    # 'washingtonpost', # 
    # 'google fact check', 
    'poynter',

    "factcheck"
]

pending_ids = []
pending_labels = []
pending_preds = []

def get_ans(result):
    fact_check_list = ["fact check", "factcheck", "fact-check"]
    for i in fact_check_list:
        if i in result["title"].lower():
            return True

        else:
            for site in websites:
                if site in result["url"].lower():
                    return True
    return False

for c, claim in enumerate(wrong_claims):
    for i in range(len(data_search)):
        if claim.strip() in data_search[i]["claim"].strip():
            results = data_search[i][f"{search_engine}_search_results"]["web"]["results"]
            ok = False
            for result in results:
                if get_ans(result):
                    ok = True
                    break
            if not ok:
                pending_ids.append(i)
                pending_labels.append(wrong_labels[c])
                pending_preds.append(wrong_preds[c])
            break


In [ ]:
pending_claims = [data_search[i]["claim"] for i in pending_ids]
len(pending_claims)


299

In [11]:
import numpy as np

df = pd.read_excel("diff_llama3_v8.xlsx", 
                   # usecols=["claim", "label", "human check", "new claim", "evidence_url"]
                   )

def update_item(data, evidence_url, index):
    flg = False
    for item in data:
        if item["id"] == index:
            flg = True
            if isinstance(evidence_url, str):
                item["evidence_url"] = evidence_url
            break
    # if not flg:
    #     print(index)
data_train = load_train()
data_type1 = load_type1()
data_type2 = load_type2()
ids = df["id"].tolist()
evidences = df["evidence_url"].tolist()

for i, index in enumerate(ids):
    update_item(data_train, evidences[i], index)
    update_item(data_type1, evidences[i], index)
    update_item(data_type2, evidences[i], index)



In [12]:
# save_type1(data_type1)
# save_type2(data_type2)

# with open("covmis_data/train.json", "w") as f:
#     json.dump(data_train, f, indent=4)

